In [13]:
import pandas as pd
from pydynpd import regression
import pathlib

In [21]:
path = pathlib.Path().resolve()

'/Users/zwanran/Desktop/ECMA31320/final/ECMA31320-project/data'

In [35]:
df = pd.read_csv(str(path).replace("code/Internal_IV", "data") + "/gvcomix_transformed.csv", na_values= '#DIV/0!')
iv = pd.read_csv(str(path).replace("code/Internal_IV", "data") + "/transport_size.csv", na_values= '#DIV/0!')

In [36]:
df = df.merge(iv)

In [38]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(df['country'])
df.drop('country', axis=1, inplace=True)
df['country'] = label
df.fillna(0, inplace=True)

In [81]:
def reg(method=" ", additional_cntrl=" ", time=" ", iv = 'outpd%'):
    # System GMM by default (nolevel)
    # No stationary controls by default
    # No time FE by default
    command_str='onset2COWCS L(1:?).onset2COWCS s6 outpd% ' + unstationary + ' ' +  additional_cntrl + ' | gmm(onset2COWCS, 2:3) gmm(s6, 2:3) iv(' + iv + ') | ' + time + ' ' + method
    mydpd = regression.abond(command_str, df, ['country', 't'])
    mydpd.models[0].regression_table

In [39]:
unstationary = 'ecgrowth logpop_M logpopdens logoutreg'
stationary = 'logmountain ethnic_fractionalization religion_fractionalization language_fractionalization leg_british opec'

In [83]:
## System GMM
reg(iv = 'avgs6')

 m1
 Dynamic panel-data estimation, two-step system GMM
 Group variable: country                           Number of obs = 2896     
 Time variable: t                                  Min obs per group: 16    
 Number of instruments = 101                       Max obs per group: 16    
 Number of groups = 181                            Avg obs per group: 16.00 
+----------------+------------+---------------------+------------+-----------+----+
|  onset2COWCS   |   coef.    | Corrected Std. Err. |     z      |   P>|z|   |    |
+----------------+------------+---------------------+------------+-----------+----+
| L1.onset2COWCS | 0.0533116  |      0.0381043      | 1.3990977  | 0.1617837 |    |
|       s6       | -0.0808097 |      0.1317100      | -0.6135428 | 0.5395175 |    |
|     outpd%     | 0.0205467  |      0.0073558      | 2.7932543  | 0.0052181 | ** |
|    ecgrowth    | -0.1715615 |      0.0622763      | -2.7548442 | 0.0058720 | ** |
|    logpop_M    | 0.0936081  |      0.0475914  

In [84]:
reg(iv = 'avgs6', time="timedumm")

 m1
 Dynamic panel-data estimation, two-step system GMM
 Group variable: country                           Number of obs = 2896     
 Time variable: t                                  Min obs per group: 16    
 Number of instruments = 117                       Max obs per group: 16    
 Number of groups = 181                            Avg obs per group: 16.00 
+----------------+------------+---------------------+------------+-----------+----+
|  onset2COWCS   |   coef.    | Corrected Std. Err. |     z      |   P>|z|   |    |
+----------------+------------+---------------------+------------+-----------+----+
| L1.onset2COWCS | 0.0435100  |      0.0331129      | 1.3139901  | 0.1888495 |    |
|       s6       | 0.1547472  |      0.2680674      | 0.5772695  | 0.5637574 |    |
|     outpd%     | 0.0251923  |      0.0102466      | 2.4586071  | 0.0139477 | *  |
|    ecgrowth    | -0.1464951 |      0.0620731      | -2.3600402 | 0.0182730 | *  |
|    logpop_M    | 0.0686170  |      0.0471095  

In [85]:
reg(iv = 'avgs6', method="nolevel")

 m3
 Dynamic panel-data estimation, two-step difference GMM
 Group variable: country                          Number of obs = 2534     
 Time variable: t                                 Min obs per group: 14    
 Number of instruments = 62                       Max obs per group: 14    
 Number of groups = 181                           Avg obs per group: 14.00 
+----------------+------------+---------------------+------------+-----------+-----+
|  onset2COWCS   |   coef.    | Corrected Std. Err. |     z      |   P>|z|   |     |
+----------------+------------+---------------------+------------+-----------+-----+
| L1.onset2COWCS | -0.3537232 |      0.0947045      | -3.7350199 | 0.0001877 | *** |
| L2.onset2COWCS | -0.4047665 |      0.0946792      | -4.2751379 | 0.0000191 | *** |
| L3.onset2COWCS | -0.4359870 |      0.1096343      | -3.9767402 | 0.0000699 | *** |
|       s6       | 0.0279461  |      0.2050866      | 0.1362650  | 0.8916118 |     |
|     outpd%     | 0.0090466  |      0.01

In [86]:
reg(iv = 'avgs6', method="nolevel", time="timedumm")

 m1
 Dynamic panel-data estimation, two-step difference GMM
 Group variable: country                          Number of obs = 2896     
 Time variable: t                                 Min obs per group: 16    
 Number of instruments = 84                       Max obs per group: 16    
 Number of groups = 181                           Avg obs per group: 16.00 
+----------------+------------+---------------------+------------+-----------+---+
|  onset2COWCS   |   coef.    | Corrected Std. Err. |     z      |   P>|z|   |   |
+----------------+------------+---------------------+------------+-----------+---+
| L1.onset2COWCS | 0.0556781  |      0.0380793      | 1.4621627  | 0.1436966 |   |
|       s6       | 0.2233013  |      0.3919025      | 0.5697878  | 0.5688216 |   |
|     outpd%     | 0.0213495  |      0.0107357      | 1.9886496  | 0.0467399 | * |
|    ecgrowth    | -0.1721811 |      0.0775839      | -2.2192889 | 0.0264671 | * |
|    logpop_M    | 0.1354702  |      0.0728434      | 1

In [82]:
## System GMM
reg()

 m2
 Dynamic panel-data estimation, two-step system GMM
 Group variable: country                          Number of obs = 2715     
 Time variable: t                                 Min obs per group: 15    
 Number of instruments = 98                       Max obs per group: 15    
 Number of groups = 181                           Avg obs per group: 15.00 
+----------------+------------+---------------------+------------+-----------+---+
|  onset2COWCS   |   coef.    | Corrected Std. Err. |     z      |   P>|z|   |   |
+----------------+------------+---------------------+------------+-----------+---+
| L1.onset2COWCS | 0.0576249  |      0.0412202      | 1.3979770  | 0.1621200 |   |
| L2.onset2COWCS | -0.0138649 |      0.0068073      | -2.0367769 | 0.0416724 | * |
|       s6       | 0.1965397  |      0.2406051      | 0.8168559  | 0.4140108 |   |
|     outpd%     | 0.0287972  |      0.0127598      | 2.2568691  | 0.0240163 | * |
|    ecgrowth    | -0.1381941 |      0.0611665      | -2.25

In [ ]:
reg(time="timedumm")

In [ ]:
## Difference GMM
reg(method="nolevel")

In [ ]:
reg(method="nolevel", time="timedumm")

# Assumption tests
## AR
- [x] We expect to reject AR(1) but not AR(2)
## Overidentification
A suggested rule of thumb is that any p-value between 0.1 and 0.25 should be fine as it is reasonably larger than a significance level of 0.05 and it is reasonably small enough to not suspect severe p-value inflation. In that case, system GMM without time dummy is the best result that we can get.